In [ ]:
import torch
from diffusers import DiffusionPipeline, LCMScheduler
import peft

In [ ]:
pipe = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", safety_checker = None)
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)

In [ ]:
#pipe.load_lora_weights("latent-consistency/lcm-lora-sdv1-5")

In [ ]:
lora_target_modules = [
        "to_q",
        "to_k",
        "to_v",
        "to_out.0",
        "proj_in",
        "proj_out",
        "ff.net.0.proj",
        "ff.net.2",
        "conv1",
        "conv2",
        "conv_shortcut",
        "downsamplers.0.conv",
        "upsamplers.0.conv",
        "time_emb_proj",
    ]
lora_config = peft.LoraConfig(
    r=64,
    target_modules=lora_target_modules,
    lora_alpha=64,
    lora_dropout=0.0,
)

unet_ = peft.get_peft_model(pipe.unet, lora_config)
unet_.load_adapter("./output/cubiCP50k", "default")

pipe.unet = unet_

In [ ]:
prompt = "architectural drawing : floor plan of a two bedroom apartment"
num_inference_steps = 8
guidance_scale = 4.0

images = []
for i in range(100) :
    img = pipe(prompt=prompt, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale, output_type="pil").images[0]
    img.save(f"./output/images/img_{i}.png")
    images.append(img)